# BAB 1C, Audio Features Scraping
Sumber: https://github.com/tashapiro/predicting-song-music-genre/tree/main/code

In [ ]:
!pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests
import shutil # to save it locally

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= 'CLIENT_ID',
                                                           client_secret= 'CLIENT_SECRET'),
                    requests_timeout=30)

In [ ]:
def get_playlist_tracks(playlist_id, n_tracks):
    data = pd.DataFrame()
    #spotify API sets a default limit of 100 tracks, to get all we need to invoke offset
    for i in range(0,n_tracks,100):
        results = sp.playlist_tracks(playlist_id,limit=100, offset=i)
        #shape json into dataframe
        track_ids = [track.get('track').get('id') for track in results.get('items')]
        df = pd.DataFrame()
        df['id'] = track_ids
        df['track'] = [track.get('track').get('name') for track in results.get('items')]
        df['artist_id'] = [track.get('track').get('artists')[0].get('id') for track in results.get('items')]
        df['artist'] = [track.get('track').get('artists')[0].get('name') for track in results.get('items')]
        df['album_id'] = [track.get('track').get('album').get('id') for track in results.get('items')]
        df['album'] = [track.get('track').get('album').get('name') for track in results.get('items')]
        df['release_date'] = [track.get('track').get('album').get('release_date') for track in results.get('items')]
        df['link'] = [track.get('track').get('href') for track in results.get('items')]
        #get audio features, merge with df
        track_features = sp.audio_features(track_ids)
        df_features = pd.DataFrame.from_dict(track_features)
        df = df.merge(df_features, on='id', how='left')
        #merge with data
        data = pd.concat([data,df])
    return data

In [ ]:
# Mendapatkan audio features untuk setiap lagu dalam playlist
df_songs = get_playlist_tracks('5v4EtazJ8p90GXyFRiIwHT', 4474)

In [ ]:
df_songs.head()

,id,track,artist_id,artist,album_id,album,release_date,link,danceability,energy,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,3pCt2wRdBDa2kCisIdHWgF,To the Bone,7d86ERlvO5UG44j7Va0Y0C,Pamungkas,3QUGmEDo4oijL93sKzlsR4,Flying Solo,2019-06-14,https://api.spotify.com/v1/tracks/3pCt2wRdBDa2...,0.668,0.727,...,0.000019,0.1000,0.939,139.991,audio_features,spotify:track:3pCt2wRdBDa2kCisIdHWgF,https://api.spotify.com/v1/tracks/3pCt2wRdBDa2...,https://api.spotify.com/v1/audio-analysis/3pCt...,344571,4
1,6Hii26x3qDErVitnGW8QtO,Runtuh,66NmU5epI0ONGmdCRbLpmW,Feby Putri,5boeKAXsTkdFlm6OnkQxQW,Runtuh,2021-10-01,https://api.spotify.com/v1/tracks/6Hii26x3qDEr...,0.593,0.287,...,0.000136,0.1190,0.191,127.783,audio_features,spotify:track:6Hii26x3qDErVitnGW8QtO,https://api.spotify.com/v1/tracks/6Hii26x3qDEr...,https://api.spotify.com/v1/audio-analysis/6Hii...,223974,4
2,2hHeGD57S0BcopfVcmehdl,Hati-Hati di Jalan,2iDVt6mFbtbDEZG5ax0dTi,Tulus,3R4IAF9ApqYeUQrv1ddyoR,Manusia,2022-03-03,https://api.spotify.com/v1/tracks/2hHeGD57S0Bc...,0.643,0.436,...,0.088900,0.1150,0.755,71.969,audio_features,spotify:track:2hHeGD57S0BcopfVcmehdl,https://api.spotify.com/v1/tracks/2hHeGD57S0Bc...,https://api.spotify.com/v1/audio-analysis/2hHe...,242000,4
3,630DpnzdfjdVqv2yLfPbAX,Bertaut,20zafXaLhm5IcXnSU93rNn,Nadin Amizah,75KyCmWatZRvlMcMRed9BG,Selamat Ulang Tahun,2020-05-28,https://api.spotify.com/v1/tracks/630DpnzdfjdV...,0.470,0.131,...,0.002450,0.1090,0.115,118.836,audio_features,spotify:track:630DpnzdfjdVqv2yLfPbAX,https://api.spotify.com/v1/tracks/630DpnzdfjdV...,https://api.spotify.com/v1/audio-analysis/630D...,315961,3
4,4GfK1qOF3uBWidbPlTCQRL,Monokrom,2iDVt6mFbtbDEZG5ax0dTi,Tulus,4szhn3xPmOJklFAcqNvTnQ,Monokrom,2016-08-03,https://api.spotify.com/v1/tracks/4GfK1qOF3uBW...,0.534,0.462,...,0.000006,0.0974,0.519,88.046,audio_features,spotify:track:4GfK1qOF3uBWidbPlTCQRL,https://api.spotify.com/v1/tracks/4GfK1qOF3uBW...,https://api.spotify.com/v1/audio-analysis/4GfK...,214567,4


In [ ]:
df_songs.shape

(4480, 25)

In [ ]:
df_songs.to_csv("audio_features.csv")

## Mengambil Data Artis

In [ ]:
def get_artist_details(artist_id):
    d = {}
    results = sp.artist(artist_id)
    d['artist_id'] = artist_id
    d['artist'] = results.get('name')
    if len(results.get('images'))>0:
        d['image'] = results.get('images')[0].get('url')
    else:
        d['image'] = ''
    d['genres'] = str(results.get('genres'))
    d['popularity'] = results.get('popularity')
    d['link'] = results.get('href')
    if len(results.get('followers'))>0:
        d['followers'] = results.get('followers').get('total')
    else:
        d['followers'] = None
    return d

In [ ]:
artists = list(set([artist for artist in df_songs['artist_id']]))

In [ ]:
artist_data = []
for artist in artists:
    data = get_artist_details(artist)
    artist_data.append(data)

In [ ]:
df_artists = pd.DataFrame(artist_data)
df_artists.head()

,artist_id,artist,image,genres,popularity,link,followers
0,12H87FnJ2Wp1MBFWAgs77M,Puti Chitara,https://i.scdn.co/image/ab6761610000e5ebd69c40...,['indonesian indie'],13,https://api.spotify.com/v1/artists/12H87FnJ2Wp...,8561
1,6gzXCdfYfFe5XKhPKkYqxV,SUPER JUNIOR,https://i.scdn.co/image/ab6761610000e5ebe9996e...,"['k-pop', 'k-pop boy group']",59,https://api.spotify.com/v1/artists/6gzXCdfYfFe...,2386680
2,49VtaZvoqBgZHQxSqlCUyp,SMTOWN,https://i.scdn.co/image/ab67616d0000b27326b7dc...,['k-pop'],31,https://api.spotify.com/v1/artists/49VtaZvoqBg...,131491
3,4xRYI6VqpkE3UwrDrAZL8L,Logic,https://i.scdn.co/image/ab6761610000e5eb6581cd...,"['conscious hip hop', 'hip hop', 'pop rap', 'r...",76,https://api.spotify.com/v1/artists/4xRYI6VqpkE...,5912837
4,31fsDbpNPKe346urriO4ma,mrld,https://i.scdn.co/image/ab6761610000e5eb0e6f5a...,['opm'],63,https://api.spotify.com/v1/artists/31fsDbpNPKe...,355751


In [ ]:
df_artists.shape

(1301, 7)

In [ ]:
df_artists.to_csv('artists.csv')